In [5]:
# n = Name of the excel file
# s = Sheet Name within the excel file
# r = Range of data which needs to be imported
# d = Data object that needs to be sent while exporting back

def import_excel(n,s,r):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    data = sheet.range(r).value
    return data 

def export_excel(n,s,r,d):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    sheet.range(r).value = d

In [6]:
data = import_excel('FRTB Data.xlsx','USDIRS','B3:D21')
#print(data)

In [7]:
#dt = Date on which operation is to be performed
#cal1 = Country of Calendar 1, For USA pass argument US
#cal2 = Country of Calendar 2, For India pass argument IN .. This is an optional argument
# At present this function suports a max of 2 calendars only for MFBD

def mfbd(dt,cal1, cal2="NIL"):
    from datetime import date
    from datetime import timedelta
    import holidays
    direction = 1
    calendar1 = holidays.country_holidays(cal1)
    if cal2 == "NIL":
        while dt.weekday() > 4 or dt in calendar1:
            if dt.month == (dt+timedelta(days=1)).month - 1:
                direction = -1
            dt = dt + timedelta(days=1)*direction 
    else:
        calendar2 = holidays.country_holidays(cal2)
        while dt.weekday() > 4 or dt in calendar1 or dt in calendar2:
            if dt.month == (dt+timedelta(days=1)).month - 1:
                direction = -1
            dt = dt + timedelta(days=1)*direction 
    return dt

In [24]:
def bootstrap(data,i=-1,p=1):
    from datetime import date,timedelta
    import holidays
    import pandas as pd
    #PricingDate = date.today() #Use todays date as the Pricing Date
    PricingDate = date(2025,5,19) # For testing currently
    SDate = mfbd(PricingDate + timedelta(days=2),'US') #Calculating the Spot Start Date
    df = pd.DataFrame(data)
    df.columns = ['Tenor','Unit','Rate']
    i = int(i)
    if i>-1:
        df.iat[i,1]=df.iat[i,1]+0.0001*p
        
    df['MatDate'] = SDate 
    df['PmtDate'] = SDate

    #Populate Maturity Date and Payment Date adjusting for pay delay

    for index,row in df.iterrows():
        if row['Unit']=='d':
            df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(days=row['Tenor']),'US')
        elif row['Unit']=='w':
            df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(weeks=row['Tenor']),'US')
        elif row['Unit']=='m':
            df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(months=row['Tenor']),'US')
        elif row['Unit']=='y':
            df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(years=row['Tenor']),'US')
        
        df.at[index,'PmtDate'] = mfbd(df.at[index,'MatDate'] + pd.offsets.DateOffset(days=2),'US') #SOFR Swaps have 2 days Payment Delay
    
    df['MatDate'] = pd.to_datetime(df['MatDate']).dt.date # To remove hh:mm:ss from Date
    df['PmtDate'] = pd.to_datetime(df['PmtDate']).dt.date
    df['DC'] = df['PmtDate'].diff(periods=1) #Compute Periodic day counts
    
    for i in range(0,15):
        df.at[i,'DC']= pd.to_timedelta(df.at[i,'PmtDate']-SDate) #Compute DC value from Start Date for tenors upto 1 year
    
    
    df['DC'] = pd.to_numeric(df['DC'].dt.days)
    df['DC']=df['DC']/360 #Since Daycount is Act/360 for USD SOFR
 
    import numpy as np
    df['DF'] = 1.000000
 
    SDate = mfbd(SDate,'US')


    #Just testing, delete later
    #for index,row in df.iterrows():
    #    df.at[index,'DF'] = 1/(pow(1+row['Rate'],row['Year']))


    #Compute actual Curve DFs for USD SOFR
    #The for loop is massively simplified to solve a linear system of equations. Will write a full document on how this is calculated later. 
    # For queries in the interim email pushkars@myyahoo.com 
    dfdccumprodsum = 0
    for index,row in df.iterrows():
        if index < 14:
            df.at[index,'DF'] = 1/(1+df.at[index,'Rate']*df.at[index,'DC'])
        else:
            df.at[index,'DF'] = (1 - df.at[index,'Rate']*dfdccumprodsum)/(1+df.at[index,'Rate']*df.at[index,'DC'])
            dfdccumprodsum = dfdccumprodsum + df.at[index,'DC']*df.at[index,'DF']

    #Compute Zeros for USD SOFR
    # Please note that zeros requires an assumption of the approach. Here we are going to try and match
    # it with that of bloomberg by assuming continuous compounding
    # Different systems implement this bit in a different manner


    df['Zero']=1.000000
    
    for index,row in df.iterrows():
        #p = pd.to_timedelta(df.at[index,'PmtDate']-df.at[index,'MatDate'])
        #t = float(p.days)/365 + df.at[index,'Year']
        t = (df.at[index,'PmtDate'] - SDate).days/365 #Note this implementation is slightly different and less accurate as its 
        #dividing by 365 to give years which will create issues especially as we cross leap years. Will update in future iterations
        #print(t)
        df.at[index,'Zero'] = np.log(1/df.at[index,'DF'])/t

    #Add 1 as Discount Factor for Spot Date in the Discount Factors Data Frame for US Rates 
    df.loc[-1] = [0.0,"w",0.0,SDate,SDate,0,1.00000,0]
    df.index = df.index+1
    df = df.sort_index()
    
        
    #print(df)
    return df

In [25]:
test=bootstrap(data)
print(test)
#This gives us the bootstrapped curve for USD SOFR which is the basis for all of our FX Discounting

    Tenor Unit      Rate     MatDate     PmtDate        DC        DF      Zero
0     0.0    w  0.000000  2025-05-21  2025-05-21  0.000000  1.000000  0.000000
1     1.0    w  0.043083  2025-05-28  2025-05-30  0.025000  0.998924  0.043658
2     2.0    w  0.043149  2025-06-04  2025-06-06  0.044444  0.998086  0.043707
3     3.0    w  0.043179  2025-06-11  2025-06-13  0.063889  0.997249  0.043718
4     1.0    m  0.043230  2025-06-23  2025-06-25  0.097222  0.995815  0.043739
5     2.0    m  0.043284  2025-07-21  2025-07-23  0.175000  0.992482  0.043720
6     3.0    m  0.043218  2025-08-21  2025-08-25  0.266667  0.988606  0.043568
7     4.0    m  0.043149  2025-09-22  2025-09-24  0.350000  0.985123  0.043421
8     5.0    m  0.042944  2025-10-21  2025-10-23  0.430556  0.981846  0.043143
9     6.0    m  0.042668  2025-11-21  2025-11-24  0.519444  0.978317  0.042788
10    7.0    m  0.042358  2025-12-22  2025-12-24  0.602778  0.975103  0.042407
11    8.0    m  0.042030  2026-01-21  2026-01-23  0.

In [83]:
def bootstrapfx(data,usdf,fcycode,divfactor=10000,i=-1,p=1):
    import pandas as pd
    import numpy as np
    from datetime import date,timedelta, datetime, time
    import math 
    df=pd.DataFrame(data)
    df.columns=['Tenor','Unit','Bid','Ask']
    spotrate= (df.at[0,'Bid'] + df.at[0,'Ask'])/2
    #df = df.drop(df.index[0]) #Drop the spot rate row
    #df = df.reset_index(drop=True) #Reset index after dropping spot rate row
   
    df['OR']=spotrate + (df['Bid'] + df['Ask'])/ (2*divfactor) #Allows flexibility to use 100 for JPY and 10,000 by default for others
    df.at[0,'OR']=spotrate  #Set the outright for spot rate row
    #Set Pricing Date
    PDate = date(2025,5,19)  #You can set it to desired pricing date
    SDate = mfbd(PDate + timedelta(days=2),'US',fcycode) #Calculating the Spot Start Date basis code recieved 
    
    #Generate Maturity Dates
    df['Date'] = SDate
    for index,row in df.iterrows():
        if row['Unit']=='d':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(days=row['Tenor']),'US',fcycode)
        elif row['Unit']=='w':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(weeks=row['Tenor']),'US',fcycode)
        elif row['Unit']=='m':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(months=row['Tenor']),'US',fcycode)
        elif row['Unit']=='y':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(years=row['Tenor']),'US',fcycode)


    #This will require us to interpolate and we are creating a float column as we cant interpolate on dates directly
    MyTime = time(0,0,0)
    ref = pd.Timestamp.combine(SDate,MyTime)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Days']=(df['Date']-ref)/timedelta(days=1)
    usdf['PmtDate'] = usdf['PmtDate'].astype("datetime64[ns]")
    usdf['Days']=(usdf['PmtDate']-datetime.combine(SDate,MyTime))/timedelta(days=1)
    #print(usdf)
    df['USDF']=np.interp(df['Days'],usdf['Days'],usdf['DF'])
    if  fcycode == 'XECB' or fcycode == 'GB' or fcycode == 'AU':
        df['FCYDF']=df['OR']*df['USDF']/spotrate
    else:
        df['FCYDF']=df['USDF']*spotrate/df['OR']    
    
    df['FCYYld']=(1/df['FCYDF']-1)*365/df['Days']
    
    df.at[0,'FCYYld'] = 0.0 #Override first yield with 0 as spot has no yield
    
    #This step overwrites the DFs and the Outrights with perturbed values if any
    #i = int(i)
    #if i>-1:
    #    df.iat[i,6]=df.iat[i,6]+0.0001*p
    #df['FCYDF']=1/(1+df['FCYYld']*df['Days']/365)
    #df['OR']=spotrate *df['USDF']/df['FCYDF']

    return df

In [84]:
#Step 1 - Generating DFs from Excel
data = import_excel('FRTB Data.xlsx','USDIRS','B3:D21')
usdf=bootstrap(data)
data = import_excel('FRTB Data.xlsx','FX','C2:f24')
eurdf=bootstrapfx(data,usdf,'XECB') #XECB is the code for TARGET Calendar
data = import_excel('FRTB Data.xlsx','FX','i2:l24')
gbpdf=bootstrapfx(data,usdf,'GB')
data = import_excel('FRTB Data.xlsx','FX','o2:r24')
jpydf=bootstrapfx(data,usdf,'JP',100) #JPY has a 100 division factor for points
print(jpydf)

    Tenor Unit      Bid      Ask         OR       Date    Days      USDF  \
0     0.0    d   156.39   156.40  156.39500 2025-05-21     0.0  1.000000   
1     1.0    d    -3.14    -3.13  156.36365 2025-05-22     1.0  0.999880   
2     1.0    w   -11.39   -11.37  156.28120 2025-05-28     7.0  0.999163   
3     2.0    w   -22.36   -22.35  156.17145 2025-06-04    14.0  0.998325   
4     3.0    w   -33.00   -32.00  156.07000 2025-06-11    21.0  0.997488   
5     1.0    m   -46.89   -45.82  155.93145 2025-06-23    33.0  0.996054   
6     2.0    m   -98.50   -97.07  155.41715 2025-07-22    62.0  0.992601   
7     3.0    m  -141.72  -140.48  154.98400 2025-08-21    92.0  0.989076   
8     4.0    m  -181.81  -178.32  154.59435 2025-09-22   124.0  0.985355   
9     5.0    m  -225.77  -225.38  154.13925 2025-10-21   153.0  0.982072   
10    6.0    m  -263.36  -263.03  153.76305 2025-11-21   184.0  0.978648   
11    7.0    m  -299.33  -299.08  153.40295 2025-12-22   215.0  0.975317   
12    8.0   

In [97]:
def get_delta_amount(row):
    # If USD is the notional currency
    if row['Not CCY'] == 'USD':
        return row['vs CCY Amt']   # the other side is non-USD
    # If USD is the vs currency
    elif row['vs CCY'] == 'USD':
        return row['CCY Amt']      # CCY Amt is non-USD
    else:
        return None  # or np.nan, or raise error

#Step 2 - Importing Trade Data from Excel
import pandas as pd
import numpy as np
data = import_excel('FRTB Data.xlsx','FXTrades','a2:h5')
tradeData=pd.DataFrame(data)
tradeData.columns=['Sr No','Pair','Not CCY','CCY Amt','vs CCY', 'Rate','vs CCY Amt','Date']
tradeData['Delta CCY']='USD'
tradeData['Delta Amt'] = tradeData.apply(get_delta_amount, axis=1)

curve_map = {
    'EURUSD': eurdf,
    'GBPUSD': gbpdf,
    'USDJPY': jpydf
}
spot={
    'EURUSD': eurdf.at[0,'OR'],
    'GBPUSD': gbpdf.at[0,'OR'],
    'USDJPY': jpydf.at[0,'OR']
}
tradeData['DF']=tradeData.apply(
    lambda r: np.interp(
        r['Date'].value,
        curve_map[r['Pair']].sort_values('Date')['Date'].astype(np.int64).values,
        curve_map[r['Pair']].sort_values('Date')['FCYDF'].values
    ), axis=1
)

tradeData['USD_Converted'] = tradeData.apply(
    lambda r: r['Delta Amt'] * spot[r['Pair']] 
              if r['Pair'] in ('EURUSD', 'GBPUSD')
              else r['Delta Amt'] / spot[r['Pair']],
    axis=1
)
tradeData['USD Delta'] = tradeData['USD_Converted'] * tradeData['DF']

grouped = tradeData.groupby('Pair')
summary = grouped['USD Delta'].sum()
#print(tradeData)
print("\nGrouped Summary:")
print(summary)

#Step 3 - Compute the capital charge

# Risk weight assumed to be 16%/sqrt(2) for major FX pairs as per BCBS guidelines
rw = 0.16 / np.sqrt(2)
x = summary.values
#print(x)

N = len(x)

# Medium correlation = 0.60
rho_med = 0.60
corr_med = np.full((N, N), rho_med)
np.fill_diagonal(corr_med, 1.0)

# High correlation = min(1.0, 1.25*rho) = 0.75
rho_high = min(1.0, 1.25*rho_med)
corr_high = np.full((N, N), rho_high)
np.fill_diagonal(corr_high, 1.0)

# Low correlation = max(0, 0.75*rho) = 0.45
rho_low = max(0, 0.75*rho_med)
corr_low = np.full((N, N), rho_low)
np.fill_diagonal(corr_low, 1.0)


# Compute scenario charges
charge_low  = rw * np.sqrt(x @ corr_low  @ x)
charge_med  = rw * np.sqrt(x @ corr_med  @ x)
charge_high = rw * np.sqrt(x @ corr_high @ x)

# Final FRTB FX charge = max of all scenarios
frtb_charge = max(charge_low, charge_med, charge_high)

charge_low  = round(charge_low)
charge_med  = round(charge_med)
charge_high = round(charge_high)
frtb_charge = round(frtb_charge)

print("\nFRTB Capital Charge (Vanilla FX Only):\n")
print(f"Low correlation charge   : {charge_low:,}")
print(f"Medium correlation charge: {charge_med:,}")
print(f"High correlation charge  : {charge_high:,}")
print(f"FRTB FX Capital Charge   : {frtb_charge:,}")


Grouped Summary:
Pair
EURUSD    1.108455e+06
GBPUSD    1.712367e+06
USDJPY   -1.282894e+06
Name: USD Delta, dtype: float64

FRTB Capital Charge (Vanilla FX Only):

Low correlation charge   : 233,458
Medium correlation charge: 218,850
High correlation charge  : 203,193
FRTB FX Capital Charge   : 233,458
